In [1]:
import pickle
import numpy as np
import torch
from LogLikelihoodTorch import log_likelihood,Optimizationloop, Initialize
with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = torch.from_numpy(X)
torch.set_default_dtype(torch.float64)


In [13]:
# Setting Up Parameters For Fitting of K Multivariate Watson Distributions

K = 7
p = 90

pi,kappa,mu = Initialize(p=p,K=K)
#%%

learning_rate = 0.4
n_iters = 2300

#torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':pi},
    {'params':kappa},
    {'params':mu}
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)

In [14]:
# Optimizing Distribution:
torch.autograd.set_detect_anomaly(False)
Pi_fit,Kappa_fit,mu_fit = Optimizationloop(X_tensor,[pi,kappa,mu],lose=log_likelihood,Optimizer=Adam,n_iters=n_iters,K=K)

epoch 1; Log-Likelihood = -5529658.767288913
epoch 101; Log-Likelihood = -6133236.251622235
epoch 201; Log-Likelihood = -6303479.638588682
epoch 301; Log-Likelihood = -6351965.390539119
epoch 401; Log-Likelihood = -6385877.956931774
epoch 501; Log-Likelihood = -6439476.791491906
epoch 601; Log-Likelihood = -6444577.165420928
epoch 701; Log-Likelihood = -6449701.187502897
epoch 801; Log-Likelihood = -6455471.435030314
epoch 901; Log-Likelihood = -6507074.490981433
epoch 1001; Log-Likelihood = -6520303.550341987
epoch 1101; Log-Likelihood = -6538924.011114539
epoch 1201; Log-Likelihood = -6567859.428573447
epoch 1301; Log-Likelihood = -6605050.326476892
epoch 1401; Log-Likelihood = -6634910.692692736
epoch 1501; Log-Likelihood = -6663329.562577668
epoch 1601; Log-Likelihood = -6691005.354614415
epoch 1701; Log-Likelihood = -6718291.946584919
epoch 1801; Log-Likelihood = -6745408.847696146
epoch 1901; Log-Likelihood = -6772476.251466932
epoch 2001; Log-Likelihood = -6799581.351426566
epoc

In [15]:
Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

pi_est = Softmax(Pi_fit)
kappa_est = Softplus(Kappa_fit)
#kappa_est = Kappa_fit
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

In [16]:
# Comparing Likelihood of pytorch vs. EM
import pickle
from WatsonDistribution import WatsonDistribution
MWD = WatsonDistribution(90)

K = 7
with open('EM.pickle','rb') as f:
    Pis_EM,kappas_EM,mus_EM,states = pickle.load(f)

#log_Likelihood_EM = sum(np.log([sum([ Pis_EM[j]* MWD.pdf(x,mus_EM[:,j],kappas_EM[j]) for j in range(K)]) for x in X.T]))
log_Likelihood_EM =  log_likelihood(X_tensor,torch.from_numpy(Pis_EM),torch.from_numpy(kappas_EM),torch.from_numpy(mus_EM),p=90,K=7)
log_Likelihood_torch = log_likelihood(X_tensor,pi_est,kappa_est,mu_est,p=90,K=7)

print("Likelihood Camparison:")
print(f"EM-Algortihem LogLikelihood: L = {log_Likelihood_EM}")
print(f"Torch Optimization of LogLikelihood: L = {log_Likelihood_torch}")

# Likelihood Camparison:
# EM-Algortihem LogLikelihood: L = 5642240.768217143

Likelihood Camparison:
EM-Algortihem LogLikelihood: L = 6469453.464885733
Torch Optimization of LogLikelihood: L = 6876754.183545258


In [19]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}: Pi = {Pis_EM[model]} , kappa = {kappas_EM[model]}")
    fig = px.bar(y=mus_EM[:,model],x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0: Pi = 0.15679143264517995 , kappa = 72.80264462628915


Model 1: Pi = 0.1537378367678051 , kappa = 62.089876938688164


Model 2: Pi = 0.13077628873636674 , kappa = 76.17904116835214


Model 3: Pi = 0.1411151156992721 , kappa = 70.94346750356566


Model 4: Pi = 0.1647789740863885 , kappa = 73.4130260903616


Model 5: Pi = 0.09973201322787797 , kappa = 86.32534755132596


Model 6: Pi = 0.15306833883710966 , kappa = 63.53188179780241


In [17]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}: Pi = {pi_est[model]} , kappa = {kappa_est[model]}")
    fig = px.bar(y=mu_est[:,model].detach().numpy(),x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0: Pi = 0.05715552927450844 , kappa = 486.97228134726237


Model 1: Pi = 0.1844281344417754 , kappa = 74.90141135578826


Model 2: Pi = 0.2095009498645683 , kappa = 61.21848693602361


Model 3: Pi = 0.16915163893496699 , kappa = 68.1510275055321


Model 4: Pi = 0.15742374998266284 , kappa = 87.99289986094709


Model 5: Pi = 0.1707447880117425 , kappa = 72.3419516679071


Model 6: Pi = 0.05159520948977568 , kappa = 883.9363996877267
